In [ ]:
from configuration import configuration;
from trajectory import trajectory;
from saddle_search import move;
import numpy as np;
import multiprocessing;
import json;

from MTP import moment_tensor_model;
from sampleQ import sample;
import torch;

n_traj = 10;

a = 3.645;
box = [a,a,a,90,90,90];
pos = [(0,0,0), (1/2,1/2,0), (1/2,0,1/2),(0,1/2,1/2)];
pos = [[p1+0.1 for p1 in p] for p in pos];
conf = configuration(box,pos,supercell=(2,2,2));
conf.set_potential(platform = 'ase', potential_id = 'Cu01.eam.alloy');

with open('model_2.json','r') as file:
    para = json.load(file);
para = [torch.tensor(p) for p in para];

sampling = sample(descriptor='mtp',para=para);
sampling.set_reference(conf);

conf.remove_atom(10);

#conf.add_atom([0.2+1/3,1/3,1/3]);
pos = conf.pos();

traj = trajectory(1,1);

saddle = move(conf);
saddle.relax(init=True);
traj.add_minimum(saddle.config);

Time_steps = 10;


for i in np.arange(Time_steps):
    print(i)
    action = sampling.sample(traj.trajectory[i], T = 0.1);
#    if(i==0):
#        action = [[-1,-1,0]]+[[0,0,0]]*30;
    traj.add_action(sampling);
    saddle = move(saddle.config, action=action, 
                  method='na',max_iter=30,threshold = 0.1, 
                  step = 0.4);
    res = saddle.step(accuracy = 0.05);
    if not res['Fail']:
        res_s = saddle.saddle(n_points = 5);
        if not res_s['Fail']:
            traj.add_saddle(res_s['saddle']);
            traj.add_minimum(saddle.config);
        else:
            traj.add_saddle(traj.trajectory[i],0,True);
            traj.add_minimum(traj.trajectory[i]);
    else:
        traj.add_saddle(traj.trajectory[i],0,True);
        traj.add_minimum(traj.trajectory[i]);
    
    traj.save('traj_'+str(n_traj)+'.json');
    reward = -traj.fail[-1]*5-(traj.E_s[-1]-traj.E_min[-2])+5;
    traj.add_reward(reward);
    traj.to_file('XDATCAR_'+str(n_traj));
#    sampling.step([traj],learning_rate = 10**-4);
    